In [9]:
%matplotlib inline  
#%matplotlib notebook
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import linear_model, preprocessing

In [10]:
ID = 2054551
np.random.seed(ID)
data=np.genfromtxt('dataset_3Body_NSC.dat',skip_header=1)

In [11]:
#pd.DataFrame(data)

In [12]:
data_mod=np.delete(data,[0,7,8,9,10,14,15,16,19,20,21],1)

In [13]:
print(data_mod[0])

[ 8.97224000e+00  7.53470000e+00  3.55802500e+01  2.59558259e-06
  9.73426482e-01  1.02204202e-05 -3.02142686e+00  3.70249403e+01
  4.09810100e+00  1.06255774e+09  1.00000000e+00]


In [14]:
np.shape(data_mod)

(100000, 11)

In [40]:
x=data_mod[:,:-1]
y=data_mod[:,-1]


'''#crop
index_0=np.where(y==0)[0]
index_1=np.where(y==1)[0]
index_2=np.where(y==2)[0]

k=6000
index=np.concatenate((index_0[:k],index_1[:k],index_2[:k]))

X_train=x[index]
y_train=y[index]
permutation =  np.random.permutation(len(y_train))
X_train =X_train[permutation]
y_train=y_train[permutation]

index=np.concatenate((index_0[k:],index_1[k:],index_2[k:]))

X_test=x[index]
y_test=y[index]
permutation = np.random.permutation(len(X_test))
X_test =X_test[permutation]
y_test=y_test[permutation]

#crop'''

index_0=np.where(y==0)[0]
index_1=np.where(y==1)[0]
index_2=np.where(y==2)[0]

k=9000
index=np.concatenate((index_0[:k],index_1[:k],index_2[:k]))

X=x[index]
Y=y[index]

permutation =  np.random.permutation(len(Y))
X=X[permutation]
Y=Y[permutation]

m_training = 10000

X_train = X[:m_training,:]
X_test = X[m_training:,:]
Y_train = Y[:m_training] 
Y_test = Y[m_training:]

print("X_train shape:", X_train.shape,"X_test shape:", X_test.shape,"||","y_train shape:",  Y_train.shape,"y_test shape:", Y_test.shape)

labels, freqs = np.unique(Y_train, return_counts=True) 
print("Labels in training dataset: ", labels)
print("Frequencies in training dataset: ", freqs)

# standardize the input matrix
# the transformation is computed on training data and then used on all the 3 sets
scaler = preprocessing.StandardScaler().fit(X_train) 

np.set_printoptions(suppress=True) 
X_train =  scaler.transform(X_train)
print ("Mean of the training input data:", X_train.mean(axis=0))
print ("Std of the training input data:",X_train.std(axis=0))

X_test = scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))

X_train shape: (10000, 10) X_test shape: (17000, 10) || y_train shape: (10000,) y_test shape: (17000,)
Labels in training dataset:  [0. 1. 2.]
Frequencies in training dataset:  [3300 3334 3366]
Mean of the training input data: [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
Std of the training input data: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean of the test input data: [ 0.00476181 -0.00069179 -0.02091939 -0.00346478  0.0144616  -0.01011483
 -0.00752668 -0.01526175  0.02221035 -0.00900701]
Std of the test input data: [1.00231215 1.00222805 0.9869867  1.02386673 1.00781215 0.91469777
 1.00602863 0.99954872 0.98655148 0.80664914]


In [41]:
'''from sklearn import linear_model, preprocessing
scaler = preprocessing.StandardScaler().fit(X_train) 

np.set_printoptions(suppress=True) 
X_train =  scaler.transform(X_train)

print ("Mean of the training input data:", X_train.mean(axis=0))
print ("Std of the training input data:",X_train.std(axis=0))

X_test = scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))'''

'from sklearn import linear_model, preprocessing\nscaler = preprocessing.StandardScaler().fit(X_train) \n\nnp.set_printoptions(suppress=True) \nX_train =  scaler.transform(X_train)\n\nprint ("Mean of the training input data:", X_train.mean(axis=0))\nprint ("Std of the training input data:",X_train.std(axis=0))\n\nX_test = scaler.transform(X_test)\nprint ("Mean of the test input data:", X_test.mean(axis=0))\nprint ("Std of the test input data:", X_test.std(axis=0))'

In [53]:
rf=RandomForestClassifier(random_state=ID,oob_score=True)#,verbose=True)
param={'max_leaf_nodes':[400],'n_estimators':[400], 'max_features':['sqrt']}
rf_cv=GridSearchCV(rf,param_grid=param, cv=4, 
                           return_train_score=True)
rf_cv.fit(X_train,Y_train)

GridSearchCV(cv=4,
             estimator=RandomForestClassifier(oob_score=True,
                                              random_state=2054551),
             param_grid={'max_features': ['sqrt'], 'max_leaf_nodes': [400],
                         'n_estimators': [400]},
             return_train_score=True)

In [54]:
all_scores=pd.DataFrame(rf_cv.cv_results_)
print("\nAll scores on the grid:","\n",
      all_scores[['param_max_leaf_nodes','mean_test_score', 'mean_train_score']])


All scores on the grid: 
   param_max_leaf_nodes  mean_test_score  mean_train_score
0                  400           0.6327            0.8717


In [55]:
rf_cv.score(X_test,Y_test)

0.6380588235294118

In [56]:

print("Best parameters set found:")
print(rf_cv.best_params_)

Best parameters set found:
{'max_features': 'sqrt', 'max_leaf_nodes': 400, 'n_estimators': 400}


In [57]:
rf_cv.score(X_train,Y_train)

0.8311